In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RAG - Developer Code Chat

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/examples/developer_code_chat/developer_code_chat.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fexamples%2Fdeveloper_code_chat%2Fdeveloper_code_chat.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/examples/developer_code_chat/developer_code_chat.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/use-cases/retrieval-augmented-generation/developer_code_chat/developer_code_chat.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Charu Shelar](https://github.com/CharulataShelar) |
|Reviewer(s) | [Erwin Huizenga](https://github.com/erwinh85) |

## Overview

This notebook showcases the development of an AI-powered learning assistant. This assistant is designed to help programmers or students, learn more about programming languages. The assistant answers users' questions (configured programming languages) using internal documents and Gemini model. It can assist end users with coding tasks, answer questions, and generate code. The solution has been built using the custom RAG approach and Gemini model (Gemini Pro 1.0). It stores the responses in BigQuery. This allows for the caching of more common queries and analytics.

## As a developer, you will learn the steps to implement the complete solution, i.e. :

1. To embed the document and create a vector search index using Vector Search (previously known as Matching Engine).
    - Upload new document in GCS bucket
    - Separate tab on the UI to allow end users to index newly added documents.
    - Python code file used here: developer_code_chatgenerate_embeddings_main.py

2. To build RAG (Retrieval-Augmented Generation) for intra document search
    - Use Gemini model to allow chat like conversation and retain the session conversation history limited to last N messages (3 previous messages in this case )
    - Answer programming questions using indexed documents.
    - Answer coding questions using the Gemini model if knowledge base does not have the relevant context/content.
    - To prevent hallucinations and maintain appropriate responses, the solution demonstrates how to guardrail the system's response to predetermined programming languages when handling user queries. List of supported programming languages can be configured in config.ini file
    - Python code file used here: developer_code_chatdeveloper_code_chatpy

4. To build chat UI interface using Gradio
5. Integrate BigQuery to save responses
    - Save the responses generated by the chatbot agent in Bigquery table for caching and further analytics by session ID: genai-github-assets.genai_data.assistant_feedback
    (Note: New session ID is created for each new gradio instance)

## GCP services used:

1. Vector Search (previously Matching Engine)
2. GenAI Model - Gemini Pro 1.0, textembedding-gecko
3. BigQuery


## Solution Design Flow

![genAI Asset Learning assistent](images/developer_code_chat.png)

### Data Ingestion Stages:
1. Developer team having access to new learning content can upload it to the GCS bucket.

2. Data Ingestion Pipeline will fetch documents from the GCS bucket (here "gs://genai-prod-vme-embedding/references") and create chunks based on the document sizes.

3. Further Data Ingestion Pipeline will get the embeddings for each page using Vertex AI Embeddings model API and index to Vector Search.


### Response Generation Stages:
1. The user starts a natural language query through a Gradio Chat User Interface (UI).

2. Intent classification is done using Gemini model. It classifies the message into one of the following intents: WELCOME, PROGRAMMING_QUESTION_AND_ANSWER, WRITE_CODE, FOLLOWUP, or CLOSE.

3. For the WRITE_CODE intent, the Gemini model is used to generate code using its coding capability.

4. For the PROGRAMMING_QUESTION_AND_ANSWER intent, custom orchestration (RAG) retrieves context relevant to the user query from Vector Search and summarises relavent contexts. If the answer is not found, the user query is routed to the Gemini Model to respond using its knowledge.

5. For the FOLLOWUP intent, such as explaining more or writing code for previous responses, the Gemini Model is used to generate responses using its code capability.

6. For the WELCOME and CLOSE intents, the Gemini model is used to generate appropriate responses.


## Getting Started

### Install Vertex AI SDK and other required packages


In [ ]:
!pip3 install --upgrade --user google-cloud-aiplatform \
langchain==0.1.13 \
pypdf==4.1.0 \
gradio==3.41.2 \
langchain-google-vertexai \
--quiet

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import required packages

In [ ]:
import configparser
import logging
import uuid

import gradio as gr
from utils.generate_embeddings import GenerateEmbeddings
from utils.intent_routing import IntentRouting
from utils.log_response_bq import LogDetailsInBQ
from utils.qna_vector_search import QnAVectorSearch
from vertexai.generative_models import GenerativeModel

### Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Update the project settings in config file

<div class="alert alert-block alert-warning">
<b>⚠️ Please do not change the configuration file name i.e. `config.ini` ⚠️</b>
</div>

In [ ]:
config_file = "config.ini"

#### [One-time] Update the settings in config file

**Note:** Some settings in `config.ini` file are are updated from this notebook. 
Additional parameters can be modified manually or using same code.

In [ ]:
config = configparser.ConfigParser()
config.read(config_file)

config.set("default", "project_id", PROJECT_ID)
config.set("default", "region", LOCATION)

with open(config_file, "w") as cf:
    config.write(cf)

## Logging Conversation Responses in BQ
- These logs can be used for caching and further analytics.
- If you don't want to store the responses in the BigQuery table, set the **flag_log_response_in_bq** flag to **False**.

#### [One-time] Update the settings in config file

In [ ]:
flag_log_response_in_bq = True  # @param {type:"boolean"}

config.set(
    "log_response_in_bq", "flag_log_response_in_bq", str(flag_log_response_in_bq)
)

with open(config_file, "w") as cf:
    config.write(cf)

### [One-time] Create BigQuery dataset and table to log conversation responses

If above **flag_log_response_in_bq** is set to False, then no need to run cells in below sesion.

In [ ]:
DATASET = "your-bq-dataset-id"  # @param {type:"string"}
TABLE = "your-bq-table-id"  # @param {type:"string"}

In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

def create_dataset(project_id: str, location: str, dataset_id: str) -> None:
    """
    Creates a new BigQuery dataset if it doesn't already exist.

    Args:
        project_id (str): The ID of the Google Cloud project.
        location (str): The geographic location where the dataset should reside.
        dataset_id (str): The ID of the dataset to create.

    Prints status messages indicating success or whether the dataset already existed.
    """
    
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project_id)

    # Check if the dataset already exists.
    try:
        client.get_dataset(dataset_id)
        print("Dataset {} already exists".format(dataset_id))
    except NotFound:
        print("Dataset {} is not found. Creating new dataset.".format(dataset_id))

        # Construct a full Dataset object to send to the API.
        dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")

        # The geographic location where the dataset should reside.
        dataset.location = location

        # Send the dataset to the API for creation.
        dataset = client.create_dataset(dataset, timeout=30)
        print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

create_dataset(PROJECT_ID, LOCATION, DATASET)

In [ ]:
def create_table(project_id: str, location: str, dataset_id: str, table_id: str) -> None:
    """
    Creates a new BigQuery table within a specified dataset if it doesn't already exist.

    Args:
        project_id (str): The ID of the Google Cloud project.
        location (str): The geographic location where the dataset resides.
        dataset_id (str): The ID of the dataset in which to create the table.
        table_id (str): The ID of the table to create.

    Defines a schema for the table with the following fields:
        * question (STRING, REQUIRED)
        * intent (STRING, REQUIRED)
        * assistant_response (STRING, REQUIRED)
        * session_id (STRING, REQUIRED)

    Prints status messages indicating success or whether the table already existed. 
    """
    
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project_id, location=location)

    # Set table_id to the ID of the table to create.
    table_id = f"{project_id}.{dataset_id}.{table_id}"

    try:
        client.get_table(table_id)
        print("Table {} already exists".format(table_id))
    except NotFound:
        print("Table {} is not found. Creating new table.".format(dataset_id))

        schema = [
            bigquery.SchemaField("question", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("intent", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("assistant_response", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("session_id", "STRING", mode="REQUIRED")
        ]

        table = bigquery.Table(table_id, schema=schema)
        table = client.create_table(table)
        print(
            "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
        )
    
create_table(PROJECT_ID, LOCATION, DATASET, TABLE)

In [ ]:
bq_table_id = f"{PROJECT_ID}.{DATASET}.{TABLE}"
config.set("log_response_in_bq", "bq_table_id", bq_table_id)

with open(config_file, "w") as cf:
    config.write(cf)

## [On-demand] Setup Vector Search for QnA

- Load new document in GSC bucket
- Setup a new Vector Search index (create document embeddings, index and deploy the index for semantic search)

**Note: This code block to be run only once. If run multiple times, it will re-create the embeddings and update the index**

In [ ]:
me_index_name = "your-vector-search-index-name"  # @param {type:"string"}
me_region = "your-vector-search-index-region"  # @param {type:"string"}

me_embedding_region = "your-gcp-bucket-region"  # @param {type:"string"}
me_embedding_dir = "your-gcp-bucket-name"  # @param {type:"string"}

In [ ]:
# Set GCP bucket name and location in config file
config.set("embedding", "me_embedding_region", me_embedding_region)
config.set("embedding", "me_embedding_dir", me_embedding_dir)

# Set Vector search index name and location in config file
config.set("embedding", "me_region", me_region)
config.set("embedding", "me_index_name", me_index_name)

with open(config_file, "w") as cf:
    config.write(cf)

### Upload the document in GCS bucket

These documents will be indexed by chunks of pages. Please review the chunk size in the config.ini file

In [ ]:
print(
    f'Upload the document to be indexed in GCS bucket folder path => gs://{config["embedding"]["me_embedding_dir"]}/{config["embedding"]["index_folder_prefix"]}'
)

### Setup Vector Search index

1. Create Vector Search index and Endpoint for Retrieval
2. Add Document Embeddings to Vector Store

In [ ]:
generate_embeddings = GenerateEmbeddings(config_file=config_file)
generate_embeddings.generate_embeddings()

## Chat interface using gradio app

#### Mount Learning Assistant app with gradio 
Now, Lets write a chat interface using Gradio that can be used for two tasks i.e. 

**A. Learning Assistant:**
1. QnA using indexed document(s) for configured programming languages. 
2. If it's not able to retrieve the answer from the first step, then the response is generated from the Gemini model memory as it can  assist end users with coding tasks, generate code and provide suggestion on errors. 

**B. Data:**
1. Select and index documents available in GCP bucket.

In [ ]:
import pandas as pd
from collections import defaultdict
from google.cloud import storage

def get_bucket_content(bucket_or_name: str, prefix: str, config_file: str = "config.ini") -> pd.DataFrame:
    """
    Retrieves details of objects from a Google Cloud Storage bucket with a specified prefix.

    Args:
        bucket_or_name (str): Name of the GCS bucket to query.
        prefix (str):  Prefix to filter objects within the bucket.
        config_file (str, optional): Path to the configuration file (default: "config.ini").
                             Assumes this file is used for GCS credentials. 

    Returns:
        pd.DataFrame:  A Pandas DataFrame containing the following columns:
            * name: Names of the objects.
            * size: Object sizes in MB.
            * type: Content type of the objects.
            * time_created: Timestamps of object creation.
            * updated: Timestamps of the last object update.
            * storage_class: Storage class of the objects.
    """
    
    config = configparser.ConfigParser()
    config.read(config_file)
    
    client = storage.Client()
    data = defaultdict(list)
    for blob in client.list_blobs(bucket_or_name=bucket_or_name, prefix=prefix):
        data["name"].append(blob.name)
        size = blob.size / 1024 / 1024
        data["size"].append(f"{size:.2f} MB")
        data["type"].append(blob.content_type)
        data["time_created"].append(str(blob.time_created)\
          .split(".", maxsplit=1)[0])
        data["updated"].append(str(blob.updated)\
          .split(".", maxsplit=1)[0])
        data["storage_class"].append(str(blob.storage_class)\
          .split(".", maxsplit=1)[0])
    df = pd.DataFrame(data=data)
    return df

In [ ]:
def create_app() -> gr.Blocks:
    """
    Initializes and configures the Gradio web interface for the Learning Assistant application.

    Key Functions:
        * Reads app configuration from 'developer_code_chatconfig.ini'.
        * Sets up logging.
        * Instantiates core components (IntentRouting, QnAVectorSearch, LogDetailsInBQ).
        * Loads language models for intent classification and chat.
        * Defines Gradio interface elements, including chatbot and feedback logging.
        * Handles user input and orchestrates responses.
        * Includes error handling to provide a graceful response in case of exceptions.

    Returns:
        gr.Blocks: The configured Gradio interface object.
        tuple: In case of errors, returns a tuple with an error message, "ERROR" string,
               and empty answer reference.
    """

    try:
        # Set up logging for the application
        logging.basicConfig(level=logging.INFO)
        logger = logging.getLogger(__name__)

        # Initialize core components using configuration settings
        genai_assistant = IntentRouting(config_file=config_file, logger=logger)
        genai_qna = QnAVectorSearch(config_file=config_file, logger=logger)
        generate_embeddings = GenerateEmbeddings(config_file=config_file)

        # Initialize logging to BigQuery (if configured)
        bq_logger = None
        if bool(config["log_response_in_bq"]["flag_log_response_in_bq"]):
            bq_logger = LogDetailsInBQ(config_file=config_file)

        # Load language models for QnA and conversational interaction
        model = GenerativeModel(config["genai_qna"]["model_name"])
        chat_model = GenerativeModel(config["genai_chat"]["model_name"])

        # Start the chat session and provide initial instructions to the chatbot
        default_programming_language = config["default"]["default_language"]
        chat = chat_model.start_chat(history=[])
        _ = chat.send_message(
            f"""You are a Programming Language Learning Assistant.
                Your task is to undersand the question and respond with the descriptive answer for the same.

                Instructions:
                1. If programming language is not mentioned, then use {default_programming_language} as default programming language to write a code.
                2. Strictly follow the instructions mentioned in the question.
                3. If the question is not clear then you can answer "I apologize, but I am not able to understand the question. Please try to elaborate and rephrase your question."

                If the question is about other programming language then DO NOT provide any answer, just say "I apologize, but I am not able to understand the question. Please try to elaborate and rephrase your question."
        """
        )

        # Define Gradio interface elements
        with gr.Blocks() as demo:
            with gr.Tab("Learning Assistant"):
                # Welcome message for the chatbot
                bot_message = "Hi there! I'm Generative AI powered Learning Assistant. I can help you with coding tasks, answer questions, and generate code. Just ask me anything you need, and I'll do my best to help!"  # pylint: disable=C0301:line-too-long

                # Generate a unique session identifier
                session_state = str(uuid.uuid4())
                logger.info("session_state : %s", session_state)

                # Configure the chatbot UI element
                chatbot = gr.Chatbot(
                    height=600,
                    label="",  # No display label for the chatbot
                    value=[[None, bot_message]],  # Initialize with the welcome message
                    avatar_images=(
                        None,
                        "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/smart_assistant/default/24px.svg",
                    ),  # Assistant avatar
                    elem_classes="message",
                    show_label=False,
                )

                msg = gr.Textbox(
                    scale=4,
                    label="",
                    placeholder="Enter your question here..",
                    elem_classes=["form", "message-row"],
                )

                def respond(message, chat_history):
                    """Sending response to gradio"""
                    # intent
                    (
                        response,
                        intent,
                        answer_reference,
                    ) = genai_assistant.classify_intent(
                        message,
                        session_state,
                        model,
                        chat_model,
                        genai_qna,
                    )

                    # append response to history
                    chat_history.append((message, response + answer_reference))

                    if bq_logger:
                        bq_logger.save_response(
                            question=message,
                            intent=intent,
                            response=response + answer_reference,
                            session_state=session_state,
                        )

                    return "", chat_history

                msg.submit(respond, [msg, chatbot], [msg, chatbot])
            with gr.Tab("Data"):
                data_df = get_bucket_content(
                    bucket_or_name=config["embedding"]["me_embedding_dir"],
                    prefix=config["embedding"]["index_folder_prefix"],
                )
                # data
                _ = gr.Dataframe(data_df)
                progress = gr.Textbox(label="Index Document Status")
                btn = gr.Button(value="Index Documents")
                btn.click(generate_embeddings.generate_embeddings, outputs=[progress])

    except Exception as e:
        import traceback

        print("Error : ", e)
        print(traceback.format_exc())

        return (
            "We're sorry, but we encountered a problem. Please try again.",
            "ERROR",
            "",
        )
    return demo

#### Launch the gradio app to view the chatbot

**Note:**
1. For a better experience, Open the demo application interface in a new tab by clicking on the Localhost url generated after running this cell.
2. For debugging mode, set `debug=True`


**Sample Questions to try on UI**
1. Where can we use python programming language?
2. What is the difference between list and set?
3. Fix the error in below code:

def create_dataset(id: str): -> None
...

SyntaxError: invalid syntax

In [ ]:
demo = create_app()
demo.queue().launch(share=True, debug=False)

### Close the demo

**Note:** Stop the previous cell to close the Gradio server running, then run this cell to free up the port utilised for running the server.

In [ ]:
demo.close()